<a href="https://colab.research.google.com/github/kutluhanNG/MachineLearning/blob/main/Generative_Adversarial_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

# Load the Fashion MNIST dataset.
# The dataset comes split into a training set and a test set.
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# Optionally, normalize the pixel values to the [0, 1] range.
X_train_full = X_train_full.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Create a validation set by splitting off a portion of the training data.
# For example, use 10% of the training data as validation.
validation_ratio = 0.1
num_validation = int(validation_ratio * X_train_full.shape[0])

X_valid = X_train_full[:num_validation]
y_valid = y_train_full[:num_validation]

X_train = X_train_full[num_validation:]
y_train = y_train_full[num_validation:]

# Display the shapes of the resulting datasets.
print("X_train shape:", X_train.shape)   # Expected: (54000, 28, 28) if using 10% for validation
print("y_train shape:", y_train.shape)
print("X_valid shape:", X_valid.shape)   # Expected: (6000, 28, 28)
print("y_valid shape:", y_valid.shape)
print("X_test shape:", X_test.shape)     # Expected: (10000, 28, 28)
print("y_test shape:", y_test.shape)


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
X_train shape: (54000, 28, 28)
y_train shape: (54000,)
X_valid shape: (6000, 28, 28)
y_valid shape: (6000,)
X_test shape: (10000, 28, 28)
y_test shape: (10000,)


In [2]:
import tensorflow as tf

codings_size = 30

Dense = tf.keras.layers.Dense

generator = tf.keras.Sequential([
    Dense(100, activation="relu", kernel_initializer="he_normal"),
    Dense(150, activation="relu", kernel_initializer="he_normal"),
    Dense(28 * 28, activation="sigmoid"),
    tf.keras.layers.Reshape([28, 28])
])

discriminator = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    Dense(150, activation="relu", kernel_initializer="he_normal"),
    Dense(100, activation="relu", kernel_initializer="he_normal"),
    Dense(1, activation="sigmoid")
])

gan = tf.keras.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False

gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [3]:
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)


In [5]:
def train_gan(gan, dataset, batch_size, codings_size, n_epochs):
  generator, discriminator = gan.layers

  for epoch in range(n_epochs):
    for X_batch in dataset:

      # Phase 1. Only the discriminator is trainable in this phase.
      noise = tf.random.normal(shape=[batch_size, codings_size])
      generated_images = generator(noise)
      X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
      y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
      discriminator.train_on_batch(X_fake_and_real, y1)

      # Phase 2. Only the generator is trainable in this phase.
      noise = tf.random.normal(shape=[batch_size, codings_size])
      y2 = tf.constant([[1.]] * batch_size)
      gan.train_on_batch(noise, y2)

train_gan(gan, dataset, batch_size, codings_size, n_epochs=10)

In [6]:
codings = tf.random.normal(shape=[batch_size, codings_size])
generated_images = generator.predict(codings)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
